Introductory information. 
Basics of how the jupyter notebook works. 

# Make a new directory and download the file

In [0]:
# Test information, show how it works

The next cell will make the directory, change to it and then display the information.  

### IMPORTANT: Change this directory name to the year you are working on!

In [1]:
# Check to see if there is anything in the directory
!ls

# Make a new directory
## CHANGE THIS
!mkdir 2018

# Change directory
## CHANGE THIS
%cd '2018'

# Make sure we have changed path
!pwd

sample_data
2018  sample_data
/content/2018
/content/2018


Then we will download the file. 

###IMPORTANT: Change the URL to reflect the file you are working on 
To do so: 

1.   Go to the [Github page](https://github.com/amanda-price/decoding-democracy) and navigate to the file you are working on. 
2.   Click on the file name.
3.   In the upper right hand corner, _right click_ the Download button and select "Copy Link Address". 
4.   Paste this address below after _wget_




In [10]:
# Change the url to the pdf you are interested in
!wget https://github.com/amanda-price/decoding-democracy/raw/master/2018/20180807_primary_precinct.pdf


#We will then display the list of files to make sure it downloaded
!ls

--2019-03-25 18:48:40--  https://github.com/amanda-price/decoding-democracy/raw/master/2018/20180807_primary_precinct.pdf
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amanda-price/decoding-democracy/master/2018/20180807_primary_precinct.pdf [following]
--2019-03-25 18:48:41--  https://raw.githubusercontent.com/amanda-price/decoding-democracy/master/2018/20180807_primary_precinct.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6325734 (6.0M) [application/octet-stream]
Saving to: ‘20180807_primary_precinct.pdf’

20180807_primary_pr 100%[===================>]   6.03M  --.-KB/s

# Load all the necessary packages and functions

In [0]:
# Install and load PDF scraper packages
!pip install PyPDF2
!pip install pdfminer.six
!pip install PyMuPDF

from PyPDF2 import PdfFileWriter, PdfFileReader
import fitz

In [0]:
def checkCandidates(candidates):
  
  # Print out the headers and make sure it matches the pdf
  print("Does this candidate list match page {}? Y or N".format(pageNum+1))
  print(candidates)
  response = input()
  
  if response.upper() == "Y":
    return candidates
  
  else:
    # We will generate a new list of candidates based on user input
    new_candidates = []
    print("How many candidates are on page {}?".format(pageNum+1))
    x = input()
    print("For each of the following questions, enter the candidates in order that they appear, left to right")
    
    for item in range(int(x)):
      print("\nFor candidate {} of {}: ".format(item+1,int(x)))
      editName = True
      
      while(editName):
        print("Please type the first name of the candidate: ")
        first = input()
        print("Please type the last name of the candidate: ")
        last = input()
        print("Does {} {} look correct? Y or N".format(first.upper(),last.upper()))
        answer = input()
        if answer.upper() == "Y":
          editName = False
          
      new_candidates.append("{} {}".format(first.upper(),last.upper()))
      
    return new_candidates

In [0]:
def scrapeOffice():
  global output
  global outfile_overall
  global prevCandidateList
  global colArray, numArray, strArray
  global pageNum
  
  prevCandidateList = []
  doc = fitz.open("20180807_primary_precinct.pdf")
  
  output = open("file_test.csv","w")
  outfile_overall = open("file_overall.csv",'w')
  

  output.write("Office,Ward,Precinct,Type of Vote,Candidate, Votes\n")
  outfile_overall.write("Office,Ward,Precinct,Type of Vote, Reg. Voters, Times Counted, Total Votes\n")
  
  # These will be defined on the first page, but needs to be initialized
  currentWard = 0
  currentPrecinct = 0
  startFlag = False
  #for i in range(doc.pageCount):
  for pageNum in range(0,doc.pageCount):
    page = doc.loadPage(pageNum) 
    text = page.getText("text")
    
    numFlag = False
    rowFlag = False
    strFlag = False
    colFlag = False
    
    
    numArray,strArray,colArray = [],[],[]

  
    lines = text.split('\n')
    for line in lines:
      row = line.strip()
      #print("new row: {}".format(row))
           
      if row.isdigit():
        colFlag = False
        numFlag = True
        #print("This is a table row: {}".format(row))
         
      if rowFlag:
        if row.startswith("CITYWIDE"):
          continue
        elif not row.startswith("W") and not row.startswith("Polling") and not row.startswith("ABSENTEE") and not row.startswith("PROVISIONAL") and not row.startswith("HOUSE") and not row.startswith("Total"):
          colFlag = True
          rowFlag = False
          pass
        
        if row.startswith("W"):
          currentWard = row[2:4]
          currentPrecinct = row[7:9]
          count = 0
        else:
          #if row.startswith("Polling"):
          #  startFlag=True
          
          #if startFlag:
          if count < 6:
            strArray.append("{},{},{}".format(currentWard,currentPrecinct,row))
            count = count+1
          #if row.startswith("Total"):        
          #  startFlag= False

      if row.startswith("Page:"):
        #print(row)
        rowFlag = True
        
      if numFlag:
        if not row == "":
          if row.endswith('%'):
            splitting = row.split(" ")
            for i in splitting:
              
              if not i.endswith("%"):
                numArray.append(i)
          elif row == "-":
            pass
          else:
            numArray.append(row)
      
      if colFlag:
        if not row == "":
          #print(row)
          if row.startswith("Times"):
            colArray.append("Times Counted")
          elif row.startswith("Counted"):
            pass
          else:
            colArray.append(row)
    print(colArray)
    if len(strArray) > 0:
      writePage()

# Run the program

In [0]:
scrapeOffice()